In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from tensorflow.python.saved_model import tag_constants

In [2]:
X_file = 'X_final.npy'
X = np.load(X_file)

In [4]:
m, n_H0, n_W0, n_C0 = X.shape

In [5]:
Y_file = 'Y_final.npy'
Y = np.load(Y_file)

In [7]:
def get_train_index(seed, train_size, m):
    np.random.seed(seed)
    train_index = np.random.choice(m, size = int(train_size*m), replace = False)
    dev_index = list(set(range(m)) - set(train_index))
    return train_index, dev_index

In [9]:
train_size = 0.9
seed = 3

train_index, dev_index = get_train_index(seed, train_size, m)

In [10]:
m_train = train_index.shape[0]

In [11]:
X_train = X[train_index]
Y_train = Y[train_index]

X_dev = X[dev_index]
Y_dev = Y[dev_index]

In [12]:
def get_predictions(X, saved_model_loc, batch_size = 32):
    graph = tf.get_default_graph()
    with tf.Session(graph = graph) as sess:
        # Restore saved values
        
        tf.saved_model.loader.load(sess, [tag_constants.SERVING], saved_model_loc)
        
        # Get restored placeholders
        inputs = graph.get_tensor_by_name('inputs_ph:0')

        # Get restored model output
        predictions = graph.get_tensor_by_name('predictions:0')

        m = X.shape[0]
        num_batches = m//batch_size + 1

        Y_preds = np.empty(shape = m)
        
        for batch in range(num_batches):
                if batch != num_batches - 1:
                    batch_index = range(batch*batch_size, (batch+1)*batch_size)
                else:
                    batch_index = range(batch*batch_size,m)
                X_batch = X[batch_index]
                Y_batch_preds = sess.run(predictions, feed_dict ={inputs: X_batch})
                Y_preds[batch_index] = Y_batch_preds
    return Y_preds

In [14]:
saved_model_loc = 'resnet50-201811220231'

Y_dev_preds_saved = get_predictions(X_dev, saved_model_loc)

INFO:tensorflow:Restoring parameters from resnet50-201811220231/variables/variables


In [16]:
Y_train_0_preds = get_predictions(np.expand_dims(X_train[0],0), saved_model_loc)

INFO:tensorflow:Restoring parameters from resnet50-201811220231/variables/variables


In [17]:
Y_train_0_preds

array([1.])

In [18]:
Y_dev_preds_saved

array([1., 1., 1., ..., 1., 1., 0.])